# Run alignment and variant calling

The parameters we use are:
    * nanopolish: 5-mer alignment, 6-mer variant calling, reverse HMM fix

In [6]:
ROOTDIR="/mnt/ebola/ebov"
import os
os.environ['PATH'] = os.getenv('PATH') + ':' + ROOTDIR + '/scripts' + ':' + ROOTDIR + '/bin/standard-RAxML'

In [7]:
%cd $ROOTDIR

/mnt/ebola/ebov


In [ ]:
%%bash

ROOTDIR="/mnt/ebola/ebov"
cd $ROOTDIR
export PATH=`pwd`/bin/bwa:$PATH
export PATH=$PATH:`pwd`/scripts
export PATH=$PATH:`pwd`/bin/nanopolish-6mer
export PATH=$PATH:`pwd`/bin/samtools
export PATH=$PATH:`pwd`/bin/marginAlign

mkdir -p analysis-EMLab
cd analysis-EMLab

echo "../models/6mer/ont_complement.pop1.model" > offset_models.fofn
echo "../models/6mer/ont_complement.pop2.model" >> offset_models.fofn
echo "../models/6mer/ont_template.model" >> offset_models.fofn

makecommands.py ../metadata/metadata.db EMLab align_defaultkmer_margin.sh | parallel -j40

In [ ]:
## After the raw files are produced, show statistics.

%cd analysis-var6mer-aln5mer-reverse-marginalign-all

In [ ]:
!makecommands.py ../metadata/metadata.db EMLab consensus.sh | parallel -j40

In [ ]:
!dump_consensus.sh ../metadata/metadata.db 20151028

In [ ]:
!rm *oct28*
!make -f ../scripts/tree.mk PREFIX=oct28 FASTA=consensus-20151028-tpr0.75.tagged.fasta THREADS=40 DIST=0.0002

In [ ]:
!make_stats_file.py ../metadata/metadata.db EMLab | intersection_vcf_stats.py /dev/stdin > tp_stats.txt

In [ ]:
!head -10 tp_stats.txt

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
stats=read.table("tp_stats.txt", header=T, sep="\t")
a = nrow(subset(stats, tag == 'np-new-filter_qual200-50' & TPR > 0.8))
subset(stats, tag == 'np-new-filter_qual200-50' & TPR < 0.2)


In [ ]:
%%R

library(ggplot2)
stats=read.table("tp_stats.txt", header=T, sep="\t")
ggplot(stats, aes(x=TPR)) + geom_histogram() + facet_wrap(~tag)